### Pcd IBR140
Vamos a resolver la cinematica directa del robot IRB140

![Alt text](imagenes\IRB140\image.png)

In [2]:
from recursos_robotica.CinematicaRec import *
import numpy as np
import sympy as sp
# Parametros de Denavit y Hartenberg para el robot IRB140
d = sp.zeros(1,6)
# d4
d[3] = 380
a = sp.zeros(1,6)
# a1
a[0] = 70
# a2
a[1] = 360
alpha = sp.Matrix([sp.rad(-90),sp.rad(0),sp.rad(90),sp.rad(-90),sp.rad(90),sp.rad(0)])
# Parametros articulares
q = sp.Matrix([sp.rad(0),sp.rad(-90),sp.rad(180),sp.rad(0),sp.rad(90),sp.rad(0)])
# Resolvemos problema directo
POSE, conf, A_i_j = pcd_IRB140(d,q,alpha,a)
# sp.pprint(A_i_j)
sp.pprint(POSE)
sp.atan2(POSE[1,0],POSE[0,0])

⎡-1  0  0   450⎤
⎢              ⎥
⎢0   1  0    0 ⎥
⎢              ⎥
⎢0   0  -1  360⎥
⎢              ⎥
⎣0   0  0    1 ⎦


pi

Para esta situacion tenemos 

Tengo la herramienta en  $450$ en $x_{0}$ y $360$ en $z_{0}$. Para la imagen tenemos $q = [-90,0,90,-90,90,0]^{T}$, Entonces nos queda $x_{6}$ queda en $-x_{0}$,  $y_{6}$ queda en $y_{0}$ y $z_{6}$ queda en $-z_{0}$.

Como $conf_{1} > 0$ brazo positivo, $conf_{2} < 0$ codo negativo y $conf_{3} > 0$ muñeca positiva(neg según el profe), 

![Alt text](imagenes\IRB140\image-1.png)

### Pci IBR140
Vamos a resolver la cinematica inverso para el IRB140

![Alt text](imagenes\IRB140\image-2.png)

In [14]:

# Parametros de Denavit y Hartenberg para el robot IRB140
d = sp.zeros(1,6)
# d4
d[3] = 380
a = sp.zeros(1,6)
# a1
a[0] = 70
# a2
a[1] = 360
alpha = sp.Matrix([sp.rad(-90),sp.rad(0),sp.rad(90),sp.rad(-90),sp.rad(90),sp.rad(0)])
# Parametros articulares
q = sp.Matrix([sp.rad(0),sp.rad(-90),sp.rad(180),sp.rad(0),sp.rad(90),sp.rad(0)])
# Resolvemos problema directo
POSE, conf, A_i_j = pcd_IRB140(d,q,alpha,a)

# Problema inverso
print("POSE")
sp.pprint(POSE)
sp.pprint(conf)
q1_actual = 0
q4_actual = 0

q = pinv_IRB140(POSE,conf,q1_actual,q4_actual,d,alpha,a)
sp.pprint(q)

POSE
⎡-1  0  0   450⎤
⎢              ⎥
⎢0   1  0    0 ⎥
⎢              ⎥
⎢0   0  -1  360⎥
⎢              ⎥
⎣0   0  0    1 ⎦
[1  -1  1]
⎡   -π         π   ⎤
⎢0  ───  π  0  ─  0⎥
⎣    2         2   ⎦


In [4]:
q = pinv_IRB140(POSE,sp.Matrix([-1,-1,1]),q1_actual,q4_actual,d,alpha,a)
sp.pprint(q.evalf())

[3.14159265358979  -3.09857653810969  2.6630046119094  3.14159265358979  2.706
0207273895  0]


En este caso al poner $conf_{1} = -1$ estoy con brazo negativo o atras, entonces el sefundo angulo me da mayor que antes, es como de la rotación inicial todavía lo tuve que rotar mas para llegar al punto destino como se ve en la figura 

![Alt text](imagenes\IRB140\image-3.png)

Ahora quiero ver si con ese q que me dio aplico nuevamente el problema directo a ver si me lleva a la POSE destino

In [5]:
POSE, conf , A_i_j = pcd_IRB140(d,q,alpha,a)
sp.pprint(POSE.evalf())
sp.pprint(conf)

⎡ -1.0     0   0.e-125  450.0⎤
⎢                            ⎥
⎢   0     1.0     0       0  ⎥
⎢                            ⎥
⎢0.e-125   0    -1.0    360.0⎥
⎢                            ⎥
⎣   0      0      0      1.0 ⎦
[-1  -1  1]


Vemos que nos da la misma POSE

## Multiples soluciones de la muñeca

solo hay que poner un cero en q5, los ejes 4 y 6 quedan alineados y $q_{4}$ y $q_{6}$ se compensan

![Alt text](imagenes/IRB140/image-4.png)


In [6]:
# PDC
q = sp.Matrix([sp.rad(0),sp.rad(-90),sp.rad(180),sp.rad(0),sp.rad(0),sp.rad(90)])
POSE, conf, A_i_j = pcd_IRB140(d,q,alpha,a)
sp.pprint(POSE.evalf())
sp.pprint(conf)

⎡ 0    0   1.0  450.0⎤
⎢                    ⎥
⎢1.0   0    0     0  ⎥
⎢                    ⎥
⎢ 0   1.0   0   360.0⎥
⎢                    ⎥
⎣ 0    0    0    1.0 ⎦
[1  -1  1]


In [7]:
# PIC
q = pinv_IRB140(POSE,conf,q1_actual,q4_actual,d,alpha,a)
sp.pprint(q)

⎡   -π            π⎤
⎢0  ───  π  0  0  ─⎥
⎣    2            2⎦


In [8]:
# Indico que mi valor actual de q4 actual es 0.1
q = pinv_IRB140(POSE,conf,q1_actual,0.1,d,alpha,a)
sp.pprint(q)

⎡   -π                     π⎤
⎢0  ───  π  0.1  0  -0.1 + ─⎥
⎣    2                     2⎦


Vemo que entre $q_{4} + q_{6}$ suman $\frac{\pi}{2}$, vamos a ver a donde vamos a parar con este q

In [9]:
q = sp.Matrix([sp.rad(0),sp.rad(-90),sp.rad(180),sp.rad(0),sp.rad(0),sp.rad(90)])
POSE, conf, A_i_j = pcd_IRB140(d,q,alpha,a)
sp.pprint(POSE.evalf())
sp.pprint(conf)

⎡ 0    0   1.0  450.0⎤
⎢                    ⎥
⎢1.0   0    0     0  ⎥
⎢                    ⎥
⎢ 0   1.0   0   360.0⎥
⎢                    ⎥
⎣ 0    0    0    1.0 ⎦
[1  -1  1]


Vemos que vamos a parar al mismo lugar, con la misma orientación, por eso es que tenemos infinitas soluciones en definitiva vamos a elegir una deacuerdo al valor de q anterior que tengamos

## Examen
El examen consistira en ver si programamos bien probando, Nos preguntaran datos puntuales de los problemas cinematicos directos e inversos del Scara y el IRB140 

1. Dada determinado POSE, ¿que variable articular para el Eje 1 se tiene?
2. Dada determinado POSE, ¿Cual es la orientación de determinado eslabon, ojo con eso ?
Esto se saca con el problema inverso, calculamos las variables articulares y despues vamos con el problema directo hasta un determinado eslabón y nos fijamos cual es la matriz R que tenemos hasta ese Elabón